# VacationPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import json

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
# Create Weather DataFrame

csv_path = os.path.join("..", "WeatherPy", "weather_data_output.csv")
weather_df = pd.read_csv(csv_path)
weather_df.dropna()
del weather_df ["Unnamed: 0"]
weather_df.head()

,City,Country,Date,Latitude,Longitude,Cloudiness (%),Humidity (%),Max Temp (F),Wind Speed (mph)
0,Yārāda,IN,2020-03-16 03:13:50,17.65,83.27,20,78,80.60,2.24
1,Hermanus,ZA,2020-03-16 03:13:52,-34.42,19.23,50,77,64.63,32.01
2,Vidim,RU,2020-03-16 03:13:55,56.41,103.11,92,91,28.47,8.81
3,Castro,BR,2020-03-16 03:13:57,-24.79,-50.01,13,86,65.79,3.42
4,City of San Pedro,PH,2020-03-16 03:13:59,14.35,121.02,75,62,86.00,5.82


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
import os
# Configure gmaps with API key.
gmaps.configure(api_key=g_key)

# Store latitude and longitude in locations and Fill NaN values and convert to float
locations = weather_df[["Latitude", "Longitude"]].astype(float)

# Store Humidity values and Fill NaN values and convert to float
humidity = weather_df["Humidity (%)"].astype(float)

In [4]:
# Plot Heatmap
fig = gmaps.figure()

# Create heat layer
heat_layer = gmaps.heatmap_layer(locations, weights=humidity, 
                                 dissipating=False, max_intensity=10,
                                 point_radius=2)

# Add layer
fig.add_layer(heat_layer)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [5]:
# Create Filtered Weather DataFrame 
ideal_weather_df = pd.DataFrame(weather_df)

# Fit Ideal Weather Conditions based on Cloudiness, Max Temp and Wind Speed
ideal_weather_df = ideal_weather_df.loc[(ideal_weather_df["Cloudiness (%)"] == 0)]                                               
ideal_weather_df = ideal_weather_df.loc[(ideal_weather_df["Max Temp (F)"] < 80) & (ideal_weather_df["Max Temp (F)"] > 70)]
ideal_weather_df = ideal_weather_df.loc[(ideal_weather_df["Wind Speed (mph)"] < 10)]    
                                              
# Drop any rows with null values and reset index
ideal_weather_df.dropna()
ideal_weather_df.reset_index(inplace=True)
del ideal_weather_df["index"]
ideal_weather_df                              

,City,Country,Date,Latitude,Longitude,Cloudiness (%),Humidity (%),Max Temp (F),Wind Speed (mph)
0,Iguape,BR,2020-03-16 03:16:41,-24.71,-47.56,0,98,71.62,1.97
1,Dakar,SN,2020-03-16 03:23:24,14.69,-17.44,0,88,71.60,6.93
2,Varadero,CU,2020-03-16 03:19:51,23.15,-81.25,0,83,71.60,5.82
3,Keetmanshoop,NaN,2020-03-16 03:25:05,-26.58,18.13,0,40,71.60,1.12
4,Mount Gambier,AU,2020-03-16 03:28:01,-37.83,140.77,0,35,73.40,9.17
5,Bilgi,IN,2020-03-16 03:32:52,16.35,75.62,0,49,78.39,7.31


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [6]:
# Create New Dataframe 
hotel_df = ideal_weather_df

#Add a "Hotel Name" column to the DataFrame
hotel_df["Hotel Name"] = ""
hotel_df

,City,Country,Date,Latitude,Longitude,Cloudiness (%),Humidity (%),Max Temp (F),Wind Speed (mph),Hotel Name
0,Iguape,BR,2020-03-16 03:16:41,-24.71,-47.56,0,98,71.62,1.97,
1,Dakar,SN,2020-03-16 03:23:24,14.69,-17.44,0,88,71.60,6.93,
2,Varadero,CU,2020-03-16 03:19:51,23.15,-81.25,0,83,71.60,5.82,
3,Keetmanshoop,NaN,2020-03-16 03:25:05,-26.58,18.13,0,40,71.60,1.12,
4,Mount Gambier,AU,2020-03-16 03:28:01,-37.83,140.77,0,35,73.40,9.17,
5,Bilgi,IN,2020-03-16 03:32:52,16.35,75.62,0,49,78.39,7.31,


In [7]:
hotel_names = []


for index, row in hotel_df.iterrows():
    # get lat, lng from df
    latitude = row["Latitude"]
    longitude = row["Longitude"]

    # geocoordinates
    target_coordinates = f"{latitude},{longitude}"
    target_search = "Hotel"
    target_radius = 5000
    target_type = "Hotel"
    
    # set up a parameters dictionary
    params = {
        "location": target_coordinates,
        "keyword": target_search,
        "radius": target_radius,
        "type": target_type,
        "key": g_key}    
    
    # base url
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # run a request using our params dictionary
    vacation_response = requests.get(base_url, params=params)

    # convert response to json
    places_data = vacation_response.json()

    print(f"Retrieving Results for Index {index}: {row['City']}.")
    
    # extract results
    results = places_data['results']
    
    try:
        print(f"Closest hotel is {results[0]['name']}.")
        hotel_df.loc[index, 'Hotel Name'] = results[0]['name']
        
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")

Retrieving Results for Index 0: Iguape.
Closest hotel is Holy Drop Skate House.
Retrieving Results for Index 1: Dakar.
Closest hotel is Radisson Blu Hotel, Dakar Sea Plaza.
Retrieving Results for Index 2: Varadero.
Closest hotel is Hotel Sol Palmeras.
Retrieving Results for Index 3: Keetmanshoop.
Closest hotel is Quiver Inn Guesthouse.
Retrieving Results for Index 4: Mount Gambier.
Closest hotel is Commodore on the Park.
Retrieving Results for Index 5: Bilgi.
Closest hotel is OYO 68887 Hotel Sangameshwar Lodging Boarding And Restaurant.


In [8]:
hotel_df

,City,Country,Date,Latitude,Longitude,Cloudiness (%),Humidity (%),Max Temp (F),Wind Speed (mph),Hotel Name
0,Iguape,BR,2020-03-16 03:16:41,-24.71,-47.56,0,98,71.62,1.97,Holy Drop Skate House
1,Dakar,SN,2020-03-16 03:23:24,14.69,-17.44,0,88,71.60,6.93,"Radisson Blu Hotel, Dakar Sea Plaza"
2,Varadero,CU,2020-03-16 03:19:51,23.15,-81.25,0,83,71.60,5.82,Hotel Sol Palmeras
3,Keetmanshoop,NaN,2020-03-16 03:25:05,-26.58,18.13,0,40,71.60,1.12,Quiver Inn Guesthouse
4,Mount Gambier,AU,2020-03-16 03:28:01,-37.83,140.77,0,35,73.40,9.17,Commodore on the Park
5,Bilgi,IN,2020-03-16 03:32:52,16.35,75.62,0,49,78.39,7.31,OYO 68887 Hotel Sangameshwar Lodging Boarding ...


In [9]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Latitude", "Longitude"]]

In [10]:
# Add marker layer ontop of heat map

markers = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig = gmaps.figure()
fig.add_layer(markers)

# Display Map
fig

Figure(layout=FigureLayout(height='420px'))